#Predictions_on_uncensored_and_censored_data
#### This notebook loads the Blodget_50k.csv data, both the bert classifier models which were build using 03_Train_BERT_Models.ipynb notebook, and both the dictionaries which were generated using 04_Train_word2vec_model_and_create_word_replacement_dictionary.ipynb notebook.

#### We classify the tweets first before replacing the words and later do the same classification on the words that replaced the swear words.

## Importing the libraries

In [ ]:

# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.8.*"
#You will use the AdamW optimizer from tensorflow/models.
!pip install -q tf-models-official==2.7.0

In [ ]:
import os
import shutil
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
from pandas.core.window.common import defaultdict
import matplotlib.pyplot as plt
import random
import numpy as np
tf.get_logger().setLevel('ERROR')
import matplotlib.pyplot as plt



# Loading the bert classifier to predict the class of tweets

In [ ]:
#Loading the Bert Classifiers which we obtained after running 03_Train_BERT_Models.ipynb notebook

model_bert_1 = tf.saved_model.load("/home/cc/faunta_bert")
model_bert_2 = tf.saved_model.load("/home/cc/davidson_bert")

#function to apply on the entire tweet text dataset
def predict(text):
  category = model_bert_1(tf.constant([text]))
  result = np.array(category)
  result = np.argmax(result)
  if result == 0:
    return "abusive"
  elif result == 1:
    return "hateful"
  elif result == 2:
    return "normal"
  else:
    return "spam"

def predict2(text):
  category = model_bert_2(tf.constant([text]))
  result = np.array(category)
  result = np.argmax(result)
  if result == 0:
    return "hate"
  elif result == 1:
    return "neither"
  elif result == 2:
    return "offensive"



In [ ]:
tweet_data = pd.read_csv("Blodget_50k.csv")

In [ ]:
tweet_data["predictions_dwmw"] = tweet_data["tweet_text"].apply(predict2)
tweet_data["predictions_fdcl"] = tweet_data["tweet_text"].apply(predict)

## Replacing swear words with replacement word by dictionary made by word2vec model trained on Google news 300m dataset and classifying using both BERT classifie

In [ ]:
with open('google_news_dict.json') as f:
    replace_dict_1 = json.load(f)

In [ ]:
def reword(text):
  sentence = text.split()
  for i in range(len(sentence)):
    if sentence[i] in replace_dict_1:
      sentence[i] = replace_dict_1[sentence[i]]
  return " ".join(sentence)


tweet_data.loc[:,'reworded_1'] = tweet_data["tweet_text"].apply(reword)

In [ ]:
prediction = tweet_data['reworded_1'].apply(predict)
prediction2 = tweet_data['reworded_1'].apply(predict2)
tweet_data.loc[:,'fdcl_predictions_on_reworded_1'] = prediction
tweet_data.loc[:,'dwmw_predictions_on_reworded_1'] = prediction2

In [ ]:
reworded_sample = tweet_data[tweet_data["tweet_text"] != tweet_data['reworded_1']]

## Replacing swear words with replacement word by dictionary made by word2vec model trained on Blodgett AAE dataset and classifying using both BERT classifier











In [ ]:
with open('Blodgett_dict.json') as f:
    replace_dict_2 = json.load(f)

In [ ]:
def reword(text):
  sentence = text.split()
  for i in range(len(sentence)):
    if sentence[i] in replace_dict_2:
      sentence[i] = replace_dict_2[sentence[i]]
  return " ".join(sentence)


tweet_data.loc[:,'reworded_2'] = tweet_data["tweet_text"].apply(reword)

In [ ]:
prediction = tweet_data['reworded_2'].apply(predict)
tweet_data.loc[:,'predictions_on_reworded_2'] = prediction

In [ ]:
prediction = tweet_data['reworded_2'].apply(predict2)
tweet_data.loc[:,'Dwmw_predictions_on_reworded_2'] = prediction

## word replacement using asterics in place of swear words and classifying using both BERT classifier

In [ ]:
with open("/home/cc/LIWC_swear_dictionary.txt") as f:
  lines = f.readlines()
  word_set = set([l.split()[0] for l in lines])

def reword5(text):
  words = text.split()
  for i in range(len(words)):
    if words[i] in word_set:
      words[i] = "*"*len(words[i])
  return " ".join(words)

In [ ]:
tweet_data.loc[:,"censored"] = tweet_data["tweet_text"].apply(reword5)

In [ ]:
tweet_data.loc[:,'fdcl_predictions_on_censored'] = tweet_data['censored'].apply(predict)
tweet_data.loc[:,'dwmw_predictions_on_censored'] = tweet_data['censored'].apply(predict2)

In [ ]:
tweet_data.to_csv("tweet_data.csv",index=False)